In [3]:
from treelib import Node, Tree

## Auxiliary Tree processing

In [18]:
li = [2,1,3,4,2,6]
pf = parkingFunc(li)

#Get unprocessed auxiliary tree
axtree = auxiliaryTree(pf)

#Map outcomeInv and input
map_outcome = {}
map_input = {}

for i in range(1, len(pf['outcome'])+1):
    map_outcome[i] = pf['outcome'][i-1]
    
for i in range(1, len(pf['input'])+1):
    map_input[i] = pf['input'][i-1]

#Get vertices of the tree excluding the root
list_of_vertices = [i for i in axtree.expand_tree(0, mode = 1)]

list_of_vertices.remove(0)

#list to store stages of the processed tree
stages = []

#create copy of original tree
ax_copy = Tree(axtree.subtree(axtree.root), deep=True)

#create copy of tree to use for the stages
tree_stage = ax_copy.subtree(0)

#process the vertices
for i in list_of_vertices:
    if ax_copy.get_node(i).is_leaf():
        continue
    tree_stage = processVertex(i, tree_stage, ax_copy, map_outcome, map_input)
    stages.append(tree_stage)

print("Original tree:")
axtree.show()
print("Processed tree:")
stages[0].show()

#Inversions

#Get path to all leaves
li_path = stages[0].paths_to_leaves()

print(li_path)

inversions = []

for i in li_path:
    n = 0
    for j in i:
        jtag = stages[0].get_node(j).tag
        for k in range(n, len(i)):
            ktag = stages[0].get_node(i[k]).tag
            #Compare tags
            if jtag > ktag:
                #Add tag pairs to list of inversions
                if [jtag,ktag] not in inversions:
                    inversions.append([jtag,ktag])
        n+=1
                
print(inversions)
print(f'Number of inversions: {len(inversions)}')

Original tree:
0
└── 6
    └── 5
        └── 4
            └── 3
                ├── 1
                └── 2

Processed tree:
0
└── 1
    └── 5
        └── 2
            └── 3
                ├── 4
                └── 6

[[0, 6, 5, 4, 3, 1], [0, 6, 5, 4, 3, 2]]
[[5, 2], [5, 3], [5, 4]]
Number of inversions: 3


## Vertex processing

In [17]:
def processVertex(vertex, stage, ogtree, map_outcome, map_input):
    
    #Get node id and tag
    n_id = ogtree.get_node(vertex).identifier
    tag = stage.get_node(n_id).tag
    
    #Create subtree with root vertex
    subtree = stage.subtree(n_id)
    
    #Get all vertices of subtree excluding the root
    vertices = [i for i in subtree.expand_tree(n_id, mode = 1)]
    vertextags = [subtree.get_node(i).tag for i in subtree.expand_tree(n_id, mode = 1)]
    
    #map vertex nid and tags
    vertex_map = {}
    
    for i in range(0, len(vertices)):
        vertex_map[vertices[i]] = vertextags[i]
    
    #(1 + kth outcome - kth preference of ogtree)
    pos = 1 + map_outcome[n_id] - map_input[n_id]
    
    #Get corresponding n_id
    vertices.sort()
    vertextags.sort()
    
    #Get corresponding tag
    if len(vertices)<pos:
        pos = len(vertices)
    
    #Get list of nids and tags
    nid_list = list(vertex_map.keys())
    tag_list = list(vertex_map.values())
    
    #Get corresponding nid
    stage_node_pos = tag_list.index(vertextags[pos-1])
    node_at_pos = nid_list[stage_node_pos]
    
    #Get corresponding tag
    stage_node = stage.get_node(node_at_pos).tag
    
    #swap tags
    stage.get_node(n_id).tag = stage_node
    stage.get_node(node_at_pos).tag = tag
    
    return stage
    

## Auxiliary Tree

In [6]:
def auxiliaryTree(pf):
    pf_copy = pf.copy()
    
    outcomeInv = pf_copy['outcomeInv']
    
    li_copy = outcomeInv.copy()
    
    tree = Tree()
    
    treeDict = {}
    
    #Create dictionary of trees with root node of corresponding outcomeInv
    for i in li_copy:
        newTree = Tree()
        newTree.create_node(i, i)
        treeDict[i] = newTree
    
    #Create tree with root node 0
    newTree = Tree()
    newTree.create_node(0, 0)
    treeDict[0] = newTree
    
    
    #Compare each element of list
    n=0
    for i in li_copy:
        flag = 0
        for j in range(n,len(li_copy)):
            if i == li_copy[j] and i == max(li_copy):
                treeDict[0].paste(0, treeDict[i])
                n+=1
                break
            elif i == li_copy[j] and li_copy.index(i) == len(li_copy)-1:
                flag = 0
            elif i == li_copy[j]:
                flag = 0
                continue
            elif i<li_copy[j]:
                #Paste tree with corresponding "parent tree"
                treeDict[li_copy[j]].paste(li_copy[j], treeDict[i])
                n+=1
                flag = 1
            elif i>li_copy[j] and j == len(li_copy)-1:
                flag = 0
                print(i)
            elif i>li_copy[j]:
                flag = 0
                continue
                
            if flag == 1:
                break
            elif flag == 0:
                #Attach entire tree to main tree with root 0
                treeDict[0].paste(0, treeDict[i])
                n+=1
                
    return treeDict[0]

## Parking Function

In [7]:
def parkingFunc(func=li, tolerance=0):
    
    li_copy = li.copy()
    length = len(li_copy)
    
    #Non-decreasing arrangement-------------
    
    li_copy.sort()
    
    #---------------------------------------
    
    num_cars_dict = dict()
    
    flag = 1
    
    pfData = {'isPF':False,
             'input':li,
              'outcome':None,
              'length':length,
             'spec':None,
             'specDetail':None,
             'nonDecArr':li_copy,
             'orderPerm':None,
             'orderPermInv':None,
              'outcomeInv':None,
             'error':None}
    
    max_tolerance = length+min(li_copy)-1
    
    if tolerance>max_tolerance:
        pfData['error'] = "Tolerance can't be greater than maximum available spot."
        return pfData

    
    for n in li_copy:
        if tolerance==0:
            if n>length:
                pfData['error'] = "Preference can't be greater than length."
                return pfData
        elif n>tolerance:
            pfData['error'] = "Preference can't be greater than tolerance."
            return pfData
        elif n<1:
            pfData['error'] = "Preference can't be zero or lesser."
            return pfData
    
    #Specification----------------------------
    
    min_val = min(li_copy)
    
    for i in range(min_val, length+min_val):
        num_cars_dict[i] = li_copy.count(i)
        
    #-----------------------------------------
    
    num_cars_dict_copy = num_cars_dict.copy()
    
    diff = 0
    
    for i in num_cars_dict_copy:
        num_cars_dict_copy[i] = num_cars_dict_copy[i] + diff
        if num_cars_dict_copy[i] >= 1:
            diff = num_cars_dict_copy[i]-1
            num_cars_dict_copy[i] = 1
            
    if diff == 0:
        flag = 1
    else:
        flag = 0
        
    outcome = []
        
    if flag == 1:
        for i in li:
            n=0
            while n==0:
                if i in outcome:
                    i+=1
                else:
                    outcome.append(i)
                    n=1
        pfData['outcome'] = outcome
        pfData['outcomeInv'] = getInverseOP(outcome)
    else:
        pass
            
    if flag == 1:
        orderperm = getOrderPerm(li_copy, li)
        inverseop = getInverseOP(orderperm)
        pfData['isPF'] = True
        pfData['spec'] = list(num_cars_dict.values())
        pfData['specDetail'] = num_cars_dict
        pfData['orderPerm'] = orderperm
        pfData['orderPermInv'] = inverseop
        return pfData
    else:
        pfData['error'] = "Invalid input."
        return pfData

## Order permutation

In [8]:
def getOrderPerm(li_copy, li):
    
    li_ = li.copy()
    li_copy_copy = li_copy.copy()
    
    length = len(li_copy_copy)
    
    orderperm = []
    
    map_ = dict()
    
    for i in range(0, length):
        map_[i+1] = li_copy_copy[i]
        
    for j in range(0, length):
        val = checkmap(map_, li_[j])
        orderperm.append(val)
                
    return orderperm

In [9]:
def checkmap(map_, val):
    for i in map_:
        if map_[i] == val:
            map_[i] = 0
            return i

## Inverse order permutation

In [10]:
def getInverseOP(orderperm):
    
    orderperm_copy = orderperm.copy()
    length = len(orderperm_copy)
    
    reference = []
    
    inverseOP = []
    
    for i in range(0, length):
        reference.append(i+1)
    
    try:
        for j in reference:
            truepos = orderperm_copy.index(j)
            pos = truepos+1
            inverseOP.append(pos)
    except:
        return None
    else:
        return inverseOP